In [1]:
import time
from ast import literal_eval
from dataclasses import dataclass, fields
import importlib

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import requests
from dataclasses import dataclass
import json

with open("../proj/config.json") as config_file:
    config = json.load(config_file)
league_url = "http://fantasy.espn.com/apis/v3/games/fba/seasons/2021/segments/0/leagues/" + str(config['league_id'])
cookies = config['cookies']

In [4]:
cookies

{'swid': '{8EB3FF23-85F1-4530-9BE7-64D12482B7EB}',
 'espn_s2': 'AECqHIgk96hNR8hAd1cr9pbXwMoLbWViGxpF%2BvAU%2Ba75ywyxIm0tcrLIeC%2Fh0W9cCQDYuQqwUtfj2w%2BSsRiGwr%2BE%2B1awm2GRTgG8eQiRdv9tum4TB2cPX8XEsIHfSBAsg7pgo9Be1lWI7Np7AmN4z1gsIoef0guDD0QdWYiYPaD90YbBEETNNJ%2BHbo17Wy9f3pg%2BuDg2VsCfz2ycOzyhseOypv%2BAIVoZVPXQvtfyWODz5nN4R%2FJipnZFlnHXJUW1mpI%3D'}

In [3]:
import bball

ModuleNotFoundError: No module named 'bball'

In [14]:
from proj.league import ESPNLeague

ModuleNotFoundError: No module named 'proj'

In [ ]:
from api import ESPN_API

In [ ]:
espn_api = ESPN_API()

In [ ]:
from classes import *


In [ ]:
for field in fields(Stat):
    print(field.name)

In [ ]:
espn_api = ESPN_API()

In [ ]:
players, name_to_pid = espn_api.get_player_info()

In [ ]:
matchup = "?view=mMatchup"
matchup_json = requests.get(league_url, cookies=cookies, params = {"view":"mMatchup"}).json()

In [ ]:
matchup_json.keys()

In [ ]:
matchup_json['gameId']

In [ ]:
matchup_json['teams'][0]

In [ ]:
home_view = "?rosterForTeamId=9&view=mDraftDetail&view=mLiveScoring&view=mMatchupScore&view=mPendingTransactions&view=mPositionalRatings&view=mRoster&view=mSettings&view=mTeam&view=modular&view=mNav"
home = requests.get(league_url+home_view).json()




In [ ]:
home.keys()

In [ ]:
scoreboard_view = "?view=mMatchup&view=mMatchupScore&view=mRoster&view=mScoreboard&view=mSettings&view=mStatus&view=mTeam&view=modular&view=mNav"
scoreboard = requests.get(league_url+scoreboard_view).json()

In [ ]:
scoreboard.keys()

In [ ]:
schedule = scoreboard['schedule']

In [ ]:
schedule[0].keys()

In [ ]:
# for obj in schedule:
#     print(obj['matchupPeriodId'])

In [ ]:
teams[5]

In [ ]:
schedule[0]['home']

In [ ]:
from typing import Dict
@dataclass
class Team:
    team_id: int
    owner: str
    abbrev: str
    name: str
    season_totals: Dict[str, float]

In [ ]:
teams = {}
for team in home['teams']:
    curr_id = team['id']
    team_obj = Team(team_id = team['id'],
                owner = team['primaryOwner'][1:-1],
                abbrev = team['abbrev'],
                name = team['nickname'],
                totals = team['valuesByStat'])
    teams[curr_id] = team_obj

In [ ]:
info = requests.get(league_url, 
                    cookies=cookies,
                    params={"view":"kona_player_info"},
                   headers={'x-fantasy-filter':
                           '{"players":{"filterStatus":{"value":["FREEAGENT","WAIVERS"]},"filterSlotIds":{"value":[0,1,2,3,4,5,6,7,8,9,10,11]},"filterRanksForScoringPeriodIds":{"value":[12]},"limit":250,"sortPercOwned":{"sortAsc":false,"sortPriority":1},"sortDraftRanks":{"sortPriority":100,"sortAsc":true,"value":"STANDARD"},"filterRanksForRankTypes":{"value":["STANDARD"]},"filterStatsForTopScoringPeriodIds":{"value":5,"additionalValue":["002021","102021","002020","012021","022021","032021","042021"]}}}',
                           'x-fantasy-platform': 'kona-PROD-a5abcf16cafe5c335041277beeafcabc2a236402',
                           'x-fantasy-source': 'kona'}
                   ).json()
for player in info['players']:
    if player['player']['lastName'] == "Barrett":
        print('sup')

In [ ]:
opponents_url = 'https://fantasy.espn.com/apis/v3/games/fba/seasons/2021/segments/0/leagues/1281614904?view=mLiveScoring&view=mMatchupScore&view=mRoster&view=mSettings&view=mStandings&view=mStatus&view=mTeam&view=modular&view=mNav'
opponents = requests.get(opponents_url, 
                        cookies=cookies, 
                        headers = {'x-fantasy-filter':
                                  '{"players":{"filterStatsForSourceIds":{"value":[0]},"filterStatsForSplitTypeIds":{"value":[5]}}}',
                                  'x-fantasy-platform': 'kona-PROD-a5abcf16cafe5c335041277beeafcabc2a236402'})


In [ ]:
opponents.json()

In [ ]:
teams = requests.get(
    "https://site.web.api.espn.com/apis/site/v2/teams?region=us&lang=en&leagues=nba",
    cookies=cookies
).json()
team_dict = {'0': 'No Team'}
for div in teams['nba']:
    for team in div['teams']:
        team_dict[team['id']] = team['shortDisplayName']

In [ ]:
team_dict

In [ ]:
total_games = requests.get('https://fantasy.espn.com/apis/v3/games/fba/seasons/2021?view=proTeamSchedules_wl',
                          cookies = cookies).json()

In [ ]:
arr_to_team_dic = {}
teams_total_games = total_games['settings']['proTeams']
for i in range(len(teams_total_games)):
    arr_to_team_dic[teams_total_games[i]['id']] = i

In [ ]:
def remaining_games(team_id):
    team = teams_total_games[arr_to_team_dic[team_id]]
    games = team['proGamesByScoringPeriod'].values()
    games_left = 0
    for game in games:
        date = game[0]['date']
        #print(date)
        if time.time()*1000 >= date:
            games_left += 1
    return (72-games_left)
        

In [ ]:
players[name_to_pid['RJ Barrett']].stats.s21_proj.avg

In [ ]:
def projections(player_name):
    player_info = players[name_to_pid[player_name]]
    team_id = player_info.nba_tid
    games_left = remaining_games(team_id)
    avg_stats = player_info.stats.s21_proj.avg
    proj_dic = {}
    for field in fields(Stat):
        proj_dic[field.name] = getattr(avg_stats, field.name) * games_left
    return proj_dic
    


In [ ]:
projections('RJ Barrett')

In [ ]:
from typing import Optional

@dataclass
class Stat:
    points: float
    turnovers: float
    minutes: float
    fgm: float
    fga: float
    ftm: float
    fta: float
    made_threes: float
    attempted_threes: float
    off_rebounds: float
    def_rebounds: float
    rebounds: float
    assists: float
    steals: float
    blocks: float
    fouls: float
        
    @property
    def fg_pct(self) -> float:
        return self.fgm/self.fga

    @property
    def ft_pct(self) -> float:
        return self.ftm/self.fta
    
    @staticmethod
    def parse_stat(stat_obj):
        def from_stat(index):
            return stat_obj[index] if index in stat_obj else None

        return Stat(
            points = from_stat('0'),
            blocks = from_stat('1'),
            steals = from_stat('2'),
            assists = from_stat('3'),
            off_rebounds = from_stat('4'),
            def_rebounds = from_stat('5'),
            rebounds = from_stat('6'),
            fouls = from_stat('9'),
            turnovers = from_stat('11'),
            fgm = from_stat('13'),
            fga = from_stat('14'),
            ftm = from_stat('15'),
            fta = from_stat('16'),
            made_threes = from_stat('17'),
            attempted_threes = from_stat('18'),
            minutes = from_stat('40')
        )

        # 7,8 = ?
        # fg_pct = 19
        # ft_pct = 20
        # 3_pct = 21
        # efg = 22
    
@dataclass
class StatDict:
    avg: Stat
    total: Stat
        
@dataclass
class Stats:
    last7: StatDict
    last15: StatDict
    last30: StatDict
    s20: StatDict
    s21: StatDict
    s21_proj: StatDict

@dataclass
class Player:
    pid: int
    name: str
    status: str # FREEAGENT, WAIVERS, etc.
    stats: Stats

In [ ]:
def get_stats(stats):
    def get_stat(stat_id):
        stat_objs = [stat for stat in stats if stat['id'] == stat_id]
        if not stat_objs:
            return None
        stat_obj = stat_objs[0]
        return StatDict(avg = Stat.parse_stat(stat_obj['averageStats']) if 'averageStats' in stat_obj else None, 
                        total= Stat.parse_stat(stat_obj['stats']))
    return Stats(
        last7 = get_stat('012021'),
        last15 = get_stat('022021'),
        last30 = get_stat('032021'),
        s20 = get_stat('002020'),
        s21 = get_stat('002021'),
        s21_proj = get_stat('102021')
    )


In [ ]:
fantasy_filter = {
    "players":
        {"filterStatus": {"value": ["FREEAGENT","WAIVERS"]},
         "filterSlotIds":{"value":[0,1,2,3,4,5,6,7,8,9,10,11]},
         "filterRanksForScoringPeriodIds":{"value":[12]},
         "limit":1000,
         "sortPercOwned":{"sortAsc":"false","sortPriority":1},
         "sortDraftRanks":{"sortPriority":100,"sortAsc":"true","value":"STANDARD"},
         "filterRanksForRankTypes":{"value":["STANDARD"]},
         "filterStatsForTopScoringPeriodIds":{"value":5,"additionalValue":["002021","102021","002020","012021","022021","032021","042021"]}
        }
}

In [ ]:
info = requests.get(league_url, 
    cookies=cookies,
    params={"view":"kona_player_info"},
   headers={'x-fantasy-filter': json.dumps(fantasy_filter),
           'x-fantasy-platform': 'kona-PROD-a5abcf16cafe5c335041277beeafcabc2a236402',
           'x-fantasy-source': 'kona'}
   ).json()

In [ ]:
for stat in player['player']['stats']:
    print(stat)

In [ ]:
free_agents = {}
name_to_pid = {}
for player in info['players']:
    player_obj = player['player']
    free_agents[player['id']] = Player(
        pid = player['id'],
        name = player_obj['fullName'],
        status = player['status'],
        stats = get_stats(player_obj['stats'])
    )
    name_to_pid[player_obj['fullName']] = player['id']


In [ ]:
#

In [ ]:
free_agents[4395625]#()

In [ ]:
get_stats(barrett_stats).last7.avg

In [ ]:
# for player in info['players']:
# #     obj = player['player']
# # #     print(obj['firstName']+' '+obj['lastName'])
# #     stats = obj['stats']
# #     if obj['lastName'] == 'Barrett':
# #         barrett_stats = stats
# #     elif obj['lastName'] == 'Hunter':
# #         hunter_stats = stats
# for stat in barrett_stats:
#     print(stat['externalId'], stat['id'], stat['stats'])
#     print('--------------')

In [ ]:
player['id']

In [ ]:
agents['schedule']